In [1]:
!pip install matplotlib
!pip install seaborn
!pip install pandas
!pip install scipy
!pip install ctgan

In [2]:
# Let's read the CSV file into a pandas DataFrame using the given local file path.
import pandas as pd

# Path to the CSV file
file_path = 'combined_synthetic_data_with_patient_ids.csv'

# Read the CSV file into a DataFrame
combined_df = pd.read_csv(file_path)

# Show the first few rows of the DataFrame
combined_df.head()


,Datetime,Code,Activity_Type,Heart rate___beats/minute,Calories burned_kcal,Exercise duration_s,Sleep type duration_minutes,Sleep duration_minutes,Floors climbed___floors,Patient ID
0,2022=12-08 14:17,NaN,No Physical Activity,72.0,NaN,NaN,NaN,NaN,NaN,User0
1,2022=12-08 21:05,LA11836-6,Running,156.0,20.0,40851.0,NaN,NaN,NaN,User0
2,2022=12-08 22:01,LA11836-6,Running,170.0,30.0,44878.0,NaN,NaN,NaN,User0
3,2022=12-08 22:06,LA11836-6,Running,166.0,20.0,21497.0,NaN,NaN,NaN,User0
4,2022=12-08 22:16,NaN,No Physical Activity,78.0,NaN,NaN,NaN,NaN,NaN,User0


In [10]:
import pandas as pd
from ctgan import CTGAN

df = combined_df

# Drop non-numeric columns or any columns that won't be synthesized
# df_cleaned = df.drop(columns=['Activity_Type','Patient ID','Code', 'Datetime','Sleep type duration_minutes','Sleep duration_minutes','Floors climbed___floors'])  # Adjust according to your needs
df_cleaned = df.drop(columns=['Activity_Type','Patient ID','Datetime'])  # Adjust according to your needs
df_cleaned['Code'] = df_cleaned['Code'].astype(str)

print(df_cleaned.head())
# Fill any missing values with the column mean or other appropriate strategies
df_cleaned.fillna(df_cleaned.mean(), inplace=True)

# Instantiate the CTGAN synthesizer
synthesizer = CTGAN()

# List of categorical columns (if any). Adjust based on your data
categorical_columns = ['Code']  # You can add more categorical columns here

# Train the CTGAN model on the data
synthesizer.fit(df_cleaned, discrete_columns=categorical_columns, epochs=300)
# synthesizer.fit(df_cleaned,  epochs=5)

# Generate synthetic data (let's say you want to generate 1000 rows)
synthetic_data = synthesizer.sample(10)

# Convert the generated data to a DataFrame
synthetic_df = pd.DataFrame(synthetic_data, columns=df_cleaned.columns)

# Save the synthetic data to a CSV file
synthetic_df.to_csv('synthetic_data_ctgan.csv', index=False)

print("Synthetic data saved to 'synthetic_data_ctgan.csv'.")


        Code  Heart rate___beats/minute  Calories burned_kcal  \
0        nan                       72.0                   NaN   
1  LA11836-6                      156.0                  20.0   
2  LA11836-6                      170.0                  30.0   
3  LA11836-6                      166.0                  20.0   
4        nan                       78.0                   NaN   

   Exercise duration_s  Sleep type duration_minutes  Sleep duration_minutes  \
0                  NaN                          NaN                     NaN   
1              40851.0                          NaN                     NaN   
2              44878.0                          NaN                     NaN   
3              21497.0                          NaN                     NaN   
4                  NaN                          NaN                     NaN   

   Floors climbed___floors  
0                      NaN  
1                      NaN  
2                      NaN  
3                 

TypeError: Could not convert ['nanLA11836-6LA11836-6LA11836-6nannan93830-893829-093829-093829-093830-893830-893829-093830-893830-893830-893830-893830-893830-893830-893830-893830-893830-893830-893830-893830-893830-893830-893830-893829-093830-893830-893830-893829-093830-893829-093830-893830-893830-8nannannannanLA11836-6nannannan93829-093830-8nanLA11836-693830-8nannannan93830-8nannannannannannan93830-8nannannannannannannannanLA11836-6nannannannannannannannanLA11836-6nanLA11836-6nanLA11836-6LA11836-6nannanLA11836-6LA11836-6LA11834-1LA11836-6LA11836-6LA11836-6LA11836-6LA11836-6LA11836-693830-8LA11836-693830-893829-093829-093830-8LA11836-693830-893830-893829-093830-893830-893829-0nannannannannanLA11836-6nanLA11836-6nanLA11836-6nannanLA11836-693830-893830-8nannannannannannannannannannannannannanLA11836-6nannannanLA11836-6nanLA11836-6nannanLA11836-6nanLA11836-6nanLA11836-6LA11834-1LA11836-6LA11836-6LA11836-6LA11836-6LA11836-6LA11836-6LA11836-693830-8LA11836-693830-893829-093830-893830-8nan9383